## Task 1: KEGG and gene id mapping

Familiarize yourself with the KEGG Rest interface and how to access it with Biopyhton:

http://www.genome.jp/kegg/rest/keggapi.html

http://nbviewer.jupyter.org/github/widdowquinn/notebooks/blob/master/Biopython_KGML_intro.ipynb

### Subtask 1.1 Extract gene lists for all (mouse) KEGG pathways and store them in a suitable Python data structure

In [611]:
import Bio
from Bio import SeqIO
from Bio.KEGG.REST import *
from Bio.KEGG.KGML import KGML_parser
from Bio.Graphics.KGML_vis import KGMLCanvas
from Bio.Graphics.ColorSpiral import ColorSpiral

from IPython.display import Image, HTML
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import StringIO
from scipy.stats import chi2_contingency, chi2, fisher_exact

In [11]:
# A bit of code that will help us display the PDF output
def PDF(filename):
    return HTML('<iframe src=%s width=700 height=350></iframe>' % filename)

# A bit of helper code to shorten long text
def head(text, lines=10):
    """ Print the first lines lines of the passed text.
    """
    print '\n'.join(text.split('\n')[:lines] + ['[...]'])

In [12]:
# Kyoto Encyclopedia of Genes and Genomes
print(kegg_info("kegg").read())

kegg             Kyoto Encyclopedia of Genes and Genomes
kegg             Release 79.0+/09-27, Sep 16
                 Kanehisa Laboratories
                 pathway     462,412 entries
                 brite       165,895 entries
                 module      377,109 entries
                 disease       1,657 entries
                 drug         10,364 entries
                 environ         850 entries
                 orthology    20,123 entries
                 genome        4,746 entries
                 genes     20,175,595 entries
                 dgenes       99,485 entries
                 compound     17,748 entries
                 glycan       10,994 entries
                 reaction     10,248 entries
                 rpair         9,661 entries
                 rclass        3,059 entries
                 enzyme        6,791 entries



In [173]:
l = kegg_list('pathway', 'mmu').read()

In [174]:
l

"path:mmu00010\tGlycolysis / Gluconeogenesis - Mus musculus (mouse)\npath:mmu00020\tCitrate cycle (TCA cycle) - Mus musculus (mouse)\npath:mmu00030\tPentose phosphate pathway - Mus musculus (mouse)\npath:mmu00040\tPentose and glucuronate interconversions - Mus musculus (mouse)\npath:mmu00051\tFructose and mannose metabolism - Mus musculus (mouse)\npath:mmu00052\tGalactose metabolism - Mus musculus (mouse)\npath:mmu00053\tAscorbate and aldarate metabolism - Mus musculus (mouse)\npath:mmu00061\tFatty acid biosynthesis - Mus musculus (mouse)\npath:mmu00062\tFatty acid elongation - Mus musculus (mouse)\npath:mmu00071\tFatty acid degradation - Mus musculus (mouse)\npath:mmu00072\tSynthesis and degradation of ketone bodies - Mus musculus (mouse)\npath:mmu00100\tSteroid biosynthesis - Mus musculus (mouse)\npath:mmu00120\tPrimary bile acid biosynthesis - Mus musculus (mouse)\npath:mmu00130\tUbiquinone and other terpenoid-quinone biosynthesis - Mus musculus (mouse)\npath:mmu00140\tSteroid hormo

In [337]:
df_path_list = pd.DataFrame([x.replace(";", "\t", 1).split("\t") for x in l.split("\n")], columns =["Pathway_ID", "Pathway_Description"])
df_path_list.set_index("Pathway_ID",inplace=True)
df_path_list.tail()

,Pathway_Description
Pathway_ID,
path:mmu05410,Hypertrophic cardiomyopathy (HCM) - Mus muscul...
path:mmu05412,Arrhythmogenic right ventricular cardiomyopath...
path:mmu05414,Dilated cardiomyopathy - Mus musculus (mouse)
path:mmu05416,Viral myocarditis - Mus musculus (mouse)
,None


In [268]:
lis = kegg_get("path:mmu00010").read().split("\n")
lis

['ENTRY       mmu00010                    Pathway',
 'NAME        Glycolysis / Gluconeogenesis - Mus musculus (mouse)',
 'DESCRIPTION Glycolysis is the process of converting glucose into pyruvate and generating small amounts of ATP (energy) and NADH (reducing power). It is a central pathway that produces important precursor metabolites: six-carbon compounds of glucose-6P and fructose-6P and three-carbon compounds of glycerone-P, glyceraldehyde-3P, glycerate-3P, phosphoenolpyruvate, and pyruvate [MD:M00001]. Acetyl-CoA, another important precursor metabolite, is produced by oxidative decarboxylation of pyruvate [MD:M00307]. When the enzyme genes of this pathway are examined in completely sequenced genomes, the reaction steps of three-carbon compounds from glycerone-P to pyruvate form a conserved core module [MD:M00002], which is found in almost all organisms and which sometimes contains operon structures in bacterial genomes. Gluconeogenesis is a synthesis pathway of glucose from noncar

In [462]:
#workaround to parse Genes per Pathway
def get_gene_per_path(l):
    bla = []
    s = 0
    e = len(lis)
    geneli = []
    mb = []
    genes = []
    for i in range(len(lis)):
        if(lis[i].startswith("GENE")):
            s = i
        elif lis[i].startswith("COMPOUND"):
            e = i
    for k in range(s,e):
        bla.append(lis[k])

    for b in range(len(bla)):
        geneli.append(bla[b].split("\t"))
    for u in range(0, len(geneli)):
        t = str(str(geneli[u]).split(";")).split(" ")
        mb.append(t)
        mb[u] = filter(None, mb[u])
    for r in range(len(mb)):
        if len(mb[r]) > 2:
            #maybe as dictionary 
            #tmp = {mb[r][2].replace("," , "").replace("\"" , "") : mb[r][1]}
            tmp = mb[r][2].replace("," , "").replace("\"" , "")
            genes.append(tmp)
    return genes

In [463]:
genes_list = []
for row in df_path_list.index[:-1]:
    lis = kegg_get(row).read().split("\n")
    print row
    genes_list.append([row, get_gene_per_path(lis)])

path:mmu00010
path:mmu00020
path:mmu00030
path:mmu00040
path:mmu00051
path:mmu00052
path:mmu00053
path:mmu00061
path:mmu00062
path:mmu00071
path:mmu00072
path:mmu00100
path:mmu00120
path:mmu00130
path:mmu00140
path:mmu00190
path:mmu00220
path:mmu00230
path:mmu00232
path:mmu00240
path:mmu00250
path:mmu00260
path:mmu00270
path:mmu00280
path:mmu00290
path:mmu00300
path:mmu00310
path:mmu00330
path:mmu00340
path:mmu00350
path:mmu00360
path:mmu00380
path:mmu00400
path:mmu00410
path:mmu00430
path:mmu00450
path:mmu00471
path:mmu00472
path:mmu00480
path:mmu00500
path:mmu00510
path:mmu00511
path:mmu00512
path:mmu00514
path:mmu00520
path:mmu00524
path:mmu00531
path:mmu00532
path:mmu00533
path:mmu00534
path:mmu00561
path:mmu00562
path:mmu00563
path:mmu00564
path:mmu00565
path:mmu00590
path:mmu00591
path:mmu00592
path:mmu00600
path:mmu00601
path:mmu00603
path:mmu00604
path:mmu00620
path:mmu00630
path:mmu00640
path:mmu00650
path:mmu00670
path:mmu00730
path:mmu00740
path:mmu00750
path:mmu00760
path:m

In [464]:
all_genes = pd.DataFrame(genes_list, columns=["Pathway_ID", "Genes"])
all_genes.set_index("Pathway_ID", inplace=True)

In [465]:
df_genes = df_path_list.join(all_genes, how="inner")
df_genes.head()

,Pathway_Description,Genes
Pathway_ID,,
path:mmu00010,Glycolysis / Gluconeogenesis - Mus musculus (m...,"[Hk2, Hk3, Hk1, Hkdc1, Gck, Gpi1, Pfkl, Pfkm, ..."
path:mmu00020,Citrate cycle (TCA cycle) - Mus musculus (mouse),"[Cs, Csl, Acly, Aco2, Aco1, Idh1, Idh2, Idh3g,..."
path:mmu00030,Pentose phosphate pathway - Mus musculus (mouse),"[Gpi1, G6pd2, G6pdx, Pgls, H6pd, Pgd, Rpe, Tkt..."
path:mmu00040,Pentose and glucuronate interconversions - Mus...,"[Gusb, Kl, Ugt2b5, Ugt1a2, Ugt1a6a, Ugt2a1, Ug..."
path:mmu00051,Fructose and mannose metabolism - Mus musculus...,"[Mpi, Pmm2, Pmm1, Gmppb, Gmppa, Gmds, Tsta3, F..."


### Subtask 1.2: Save the KEGG gene sets as a gmt file after you made sure they have the proper gene ids with respect to your DE analysis

hints: 

http://biopython.org/wiki/Annotate_Entrez_Gene_IDs

http://www.broadinstitute.org/cancer/software/gsea/wiki/index.php/Data_formats

In [467]:
df_genes.to_csv("/home/engelha/Documents/Praktikum/Integrated_Bioinformatics/notebooks/DayY_KEGG_genes.csv", sep="\t")

## Task 2: Gene Set Enrichment

### Subtask 2.1: Read in the csv file you produced during the Differential Expression module, extract a gene list (as a python list of gene symbols) from your favorite multiple correction column (and store it in a variable)

In [521]:
DE = pd.read_csv("/home/engelha/Documents/Praktikum/Integrated_Bioinformatics/notebooks/Diff_Genes_Log2FC_INDEX.csv", index_col=0)

In [522]:
DE.index.names = ["Gene_ID"]
DE.head()

,Log2_FoldChange,MannWhit_P_values,bonferroni_p_Values_corrected,sidak_p_Values_corrected,holm-sidak_p_Values_corrected,holm_p_Values_corrected,simes-hochberg_p_Values_corrected,hommel_p_Values_corrected,fdr_bh_p_Values_corrected,fdr_by_p_Values_corrected,fdr_tsbh_p_Values_corrected,fdr_tsbky_p_Values_corrected
Gene_ID,,,,,,,,,,,,
Igh,0.036984,3.455967e-08,7.545759e-04,7.542913e-04,6.963077e-04,6.965502e-04,6.965157e-04,6.269470e-04,4.488851e-07,4.744026e-06,1.984149e-07,2.107534e-07
Hcfc2,0.019614,8.087184e-05,1.000000e+00,8.289547e-01,7.406464e-01,1.000000e+00,5.000000e-01,5.000000e-01,3.424985e-04,3.619682e-03,1.513902e-04,1.608044e-04
Ccdc112,0.077647,4.945441e-12,1.079788e-07,1.079798e-07,1.063428e-07,1.063418e-07,1.063418e-07,1.041658e-07,3.244404e-10,3.428836e-09,1.434082e-10,1.523261e-10
EG635895,0.065744,4.284527e-07,9.354836e-03,9.311218e-03,8.276260e-03,8.310697e-03,8.310697e-03,6.995775e-03,3.837094e-06,4.055219e-05,1.696061e-06,1.801531e-06
Srp72,0.000394,3.359962e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.000000e-01,5.000000e-01,3.750675e-01,1.000000e+00,1.657862e-01,1.760957e-01


In [728]:
sidaks = pd.Series(DE.index)
sidaks.head()

0         Igh
1       Hcfc2
2     Ccdc112
3    EG635895
4       Srp72
Name: Gene_ID, dtype: object

In [729]:
pathway_genes = []
for row in df_genes.index:
    pathway_genes = set(pathway_genes).union(set(df_genes.loc[row]["Genes"]))
len(pathway_genes)

9128

In [900]:
shared_genes = set(sidaks).intersection(pathway_genes)
len(shared_genes)

7061

In [957]:
crossDF_DE = DE.copy()
crossDF_DE.drop(crossDF_DE.columns[0:], axis=1, inplace=True)
crossDF_DE["Diff_Exp"] = False


In [958]:
r1 = list(df_genes.loc["path:mmu00510"]["Genes"])

In [959]:
crossDF_DE.Diff_Exp.loc[shared_genes] = True

In [971]:
crossDF_Path = pd.DataFrame([pathway_genes])
crossDF_Path = crossDF_Path.T
crossDF_Path.set_index(0, inplace=True)
crossDF_Path.drop("",axis=0, inplace=True)
crossDF_Path["path:mmu00010"] = False
crossDF_Path["path:mmu00010"].loc[r1] = True

In [970]:
cross_Joined = crossDF_DE.join(crossDF_Path, how="outer")
cross_Joined.replace(np.nan, False, inplace=True)

### Subtask 2.2: Perform gene set enrichment (Fisher's exact test or an hypergeometric test will do for our purposes) with the KEGG gene sets you extracted in Task 1 (you may want to store the results in a pandas dataframe and write them to csv)

hint:

https://genetrail2.bioinf.uni-sb.de/help?topic=set_level_statistics

In [974]:
cross_table = pd.crosstab(cross_Joined.Diff_Exp, cross_Joined["path:mmu00010"], margins=True)
cross_table

path:mmu00010,False,True,All
Diff_Exp,,,
False,16838,1,16839
True,7013,48,7061
All,23851,49,23900


### Subtask 2.3: Extract a list of significantly (at 0.05 significance) enriched KEGG pathways

## Task 3: KEGG map visualization

#### hint:

http://nbviewer.jupyter.org/github/widdowquinn/notebooks/blob/master/Biopython_KGML_intro.ipynb

#### remark:

In real life you may want to use the R-based tool pathview: https://bioconductor.org/packages/release/bioc/html/pathview.html (if you insist you can also try to use r2py for using pathview from Python during the practical)

For Python (in addition to the Biopyhton module) https://github.com/idekerlab/py2cytoscape in combination with https://github.com/idekerlab/KEGGscape may be another alternative (in the future)

Generally speaking, it is always a good idea to pay attention also to other pathway databases like Reactome or WikiPathways ...

### Subtask 3.1: Pick some significantly enriched KEGG pathways of your choice from 2.3 and visualize them

### Subtask 3.2: Define a a suitable binary color scheme respresenting the fact whether a gene is significantly expressed or not

hint: 

http://www.rapidtables.com/web/color/RGB_Color.htm

### Subtask 3.3: Visualize the pathway(s) from 3.1 in such a way that the included genes have the corresponding color from 3.2 ( you may need to define a suitable mapping from single genes to what is actually shown in the pathway map...)

### Subtask 3.4: Define a suitable continuous color range representing the log2 fold changes of the all the genes in your data

hint:

http://bsou.io/posts/color-gradients-with-python

### Subtask 3.5: Visualize the pathway(s) from 3.1 in such a way that the included genes have the corresponding color from 3.4